## Compute execution ratio using dedicated service(F#)

### Overview
Tick count enriched with trade count can be used as an indicator to illustrate market activity and measure market quality by:
* computing the technical transactions/transmissions count on a tick by tick basis, *tick count*,
* comparing it to the total *trade count*

Correlated with a set of other indicators such as traded volume and price volatility, over the same look back period and the same sampling granularity, it repserents a good measurement of the market quality.

This samples enables to retrieve **on-demand** tick count data points (L1 and L1L2) by calling a dedicated service.

### Inputs/outputs
Execution ratio sample requires instrument's identifier, date time intervals and time granularity as per inputs. It returns the tick count Vs trade count sampled using the input time granularity.

This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

<div class="alert alert-block alert-info">
    <b>Note:</b> Similar scenarios are available and enable to enrich tick count with price volatility or trade sizes.
</div>

### Services used
This sample uses *gRPC requests* in order to retrieve tick by tick data from the dedicated hosted service. The queried endpoint in this script are:
* *TopologiesService*: to directly retrieve tick by tick data data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Topology.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run tick count sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.38.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Topology.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *TopologiesService*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Topology request parameters

#### 3.1 Instrument selection

In [ ]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Execution ratio granularity

In [ ]:
let granularity = TopologyGranularity.Daily

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate Trades request
let requestL1 = 
    new TopologiesRequest(
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker), 
        Granularity = granularity,
        Level = Level.Trades
    )

// Generate Trades and Book request
let requestL2 = 
    new TopologiesRequest (
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker), 
        Granularity = granularity,
        Level = Level.TradesAndBook
    )

In [ ]:
// Instantiate the topologies service
let service = new TopologiesService.TopologiesServiceClient(channel)        

#### 3.3 Retrieve data

In [ ]:
// Process Trades request
let replyL1 = service.Topologies(requestL1, headers)
 
// Retrieve topologies service response for Trades
let topologiesL1 = replyL1.Entries
let totalTicksL1 = topologiesL1 |> Seq.sumBy (fun t -> t.TicksCount)

topologiesL1 |> display

In [ ]:
// Process Trades and Book request
let replyL2 = service.Topologies(requestL2, headers)
 
// Retrieve topologies service response for Trades and Book
let topologiesL2 = replyL2.Entries
let totalTicksL2 = topologiesL2 |> Seq.sumBy (fun t -> t.TicksCount)

// Store tick count starting from the date of your choice
topologiesL2 |> display

### Step4: Visualize data

#### 4.1 Retrieve tick counts
The following code snippet retrieves *Trades ticks* and *Trades and Book ticks* counts:

In [ ]:
// Extract ticks count for Trades
let trades        = topologiesL1 |> Seq.map (fun t -> (new DateTime(t.Begin.Year, t.Begin.Month, t.Begin.Day), t.TicksCount))
// Extract the price Trades and Book
let tradesAndBook = topologiesL2 |> Seq.map (fun t -> (new DateTime(t.Begin.Year, t.Begin.Month, t.Begin.Day), t.TicksCount))

#### 4.2 Compute execution ratio
The following code snippet computes execution ratio by dividing  *Trades ticks* and *Trades and Book ticks* counts:

In [ ]:
let computeExecutionRatioPercent (d1,tradeCount) (d2,tickCount) =
    (d1, (double tradeCount ) / (double tickCount) * 100.)

let executionRatio = Seq.map2 computeExecutionRatioPercent trades tradesAndBook


#### 4.2 Plot execution ratio

In [ ]:
let executionRatiochart =
    [
        new Scattergl(
            name = "Trades ticks count",
            mode = "lines",
            x = (trades |> Seq.map fst),
            y = (trades |> Seq.map snd)
        )
        new Scattergl(
            name = "Trades and Book ticks count",
            mode = "lines",
            x = (tradesAndBook |> Seq.map fst),
            y = (tradesAndBook |> Seq.map snd)
        )
        new Scattergl(
            name = "Execution ratio Percent",
            mode = "lines",
            x = (executionRatio |> Seq.map fst),
            y = (executionRatio |> Seq.map snd),
            yaxis = "y2"
        )
    ]


let layout = 
    new Layout.Layout( 
        title = $"Execution ratio for {ticker}",
        xaxis = new Xaxis( title = "Time"),
        yaxis = new Yaxis (
            title = "Ticks count", 
            autorange = true,
            side = "left"
        ),
        yaxis2 = new Yaxis (
            title = "Ratio", 
            autorange = true,
            side = "right"
        )
    )

Chart.Plot(executionRatiochart, layout)
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display